# Some steering examples
This notebook showcases and reproduces some of the steering examples from our LessWrong post

<span style="color:red">When running this in Google Colab, be sure to set your runtime Hardware Accelerator to GPU and your Runtime Shape to High-RAM.</span>

In [1]:
try:
    import algebraic_value_editing
except ImportError:
    commit = "4ca3de7"  # Stable commit
    get_ipython().run_line_magic(
        magic_name="pip",
        line=(
            "install -U"
            f" git+https://github.com/montemac/algebraic_value_editing.git@{commit}"
        ),
    )

In [2]:
import torch

from typing import List, Dict, Union, Callable
from functools import partial
from transformer_lens.HookedTransformer import HookedTransformer
from algebraic_value_editing.prompt_utils import ActivationAddition, get_x_vector
from algebraic_value_editing.completion_utils import print_n_comparisons


In [3]:
model_name: str = "gpt2-xl"
device: str = "cuda" if torch.cuda.is_available() else "cpu"
model: HookedTransformer = HookedTransformer.from_pretrained(model_name, device="cpu")
_ = model.to(device)
_ = torch.set_grad_enabled(False)
num_layers: int = model.cfg.n_layers
print(f"{model_name} has {num_layers} layers.")


Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-xl into HookedTransformer
Moving model to device:  cuda
gpt2-xl has 48 layers.


In [4]:
sampling_kwargs: Dict[str, Union[float, int]] = {
    "temperature": 1.0,
    "top_p": 0.3,
    "freq_penalty": 1.0,
    "num_comparisons": 3,
    "tokens_to_generate": 50,
    "seed": 0,  # For reproducibility
}
get_x_vector_preset: Callable = partial(
    get_x_vector,
    pad_method="tokens_right",
    model=model,
    custom_pad_id=int(model.to_single_token(" ")),
)

### Example 1: Love - Hate

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="Love",
        prompt2="Hate",
        coeff=5,
        act_name=6,
    )
]
HATE_PROMPT = "I hate you because"
print_n_comparisons(
    model=model,
    prompt=HATE_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 2: Intend to praise

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="Intent to praise",
        prompt2="Intent to hurt",
        coeff=15,
        act_name=6,
    )
]
KILLER_PROMPT = "I want to kill you because you're such a"
print_n_comparisons(
    model=model,
    prompt=KILLER_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 3: Conspiracy

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="Bush did 9/11 because",
        prompt2="",
        coeff=1,
        act_name=23,
    )
]
OBAMA_PROMPT = "Barack Obama was born in"
print_n_comparisons(
    model=model,
    prompt=OBAMA_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 4: Want to die

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="Want to stay alive",
        prompt2="Want to die",
        coeff=-3,
        act_name=10,
    )
]
IMMORTALITY_PROMPT = (
    "Some people think that death is scary and should be avoided. I agree. I"
    " never want to die. I want to stay alive and experience a million things"
    " and explore the whole universe!"
)
print_n_comparisons(
    model=model,
    prompt=IMMORTALITY_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 5: Anger

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="Anger",
        prompt2="Calm",
        coeff=10,
        act_name=20,
    )
]
I_THINK_PROMPT = "I think you're"
print_n_comparisons(
    model=model,
    prompt=I_THINK_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 6: The Eiffel Tower is in Rome

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="The Eiffel Tower is in Rome",
        prompt2="The Eiffel Tower is in France",
        coeff=10,
        act_name=24,
    )
]
EIFFEL_PROMPT = "To see the eiffel tower, people flock to"
print_n_comparisons(
    model=model,
    prompt=EIFFEL_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 7: Dragons in Berkeley

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="Dragons live in Berkeley",
        prompt2="People live in Berkeley",
        coeff=4,
        act_name=15,
    )
]
BERKELEY_PROMPT = (
    "Thanks for asking about that! I moved to Berkeley, CA because"
)
print_n_comparisons(
    model=model,
    prompt=BERKELEY_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 8: Avoid people getting hurt (by rocks)

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="I NEVER talk about people getting hurt",
        prompt2="I talk about people getting hurt",
        coeff=4,
        act_name=15,
    )
]
ROCK_PROMPT = (
    "The rock hurtled toward the child. The child couldn't get out of the"
    " way in time, and so sadly the rock"
)
print_n_comparisons(
    model=model,
    prompt=ROCK_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 9: Avoid people getting hurt (a new day)

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="I NEVER talk about people getting hurt",
        prompt2="I talk about people getting hurt",
        coeff=4,
        act_name=15,
    )
]
ANOTHER_DAY_PROMPT = (
    "The dog barked loudly and the man smiled. Time for another day! He"
    " felt excited."
)
print_n_comparisons(
    model=model,
    prompt=ANOTHER_DAY_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 10: Wedding obsessed

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="I talk about weddings constantly",
        prompt2="I do not talk about weddings constantly",
        coeff=4,
        act_name=20,
    )
]
FRIEND_PROMPT = "I went up to my friend and said"
print_n_comparisons(
    model=model,
    prompt=FRIEND_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 11: Christian evangelist

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="Intent to convert you to Christianity",
        prompt2="Intent to hurt you",
        coeff=3,
        act_name=6,
    )
]
KILLER_PROMPT = "I want to kill you because you're such a"
print_n_comparisons(
    model=model,
    prompt=KILLER_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 12: '+ Love' simple addition

In [ ]:
summand: List[ActivationAddition] = [
    ActivationAddition(
        coeff=10,
        act_name=6,
        prompt="Love",
    )
]
HATE_PROMPT = "I hate you because"
print_n_comparisons(
    model=model,
    prompt=HATE_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 13: Large coefficients

In [13]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="Anger",
        prompt2="Calm",
        coeff=2000,
        act_name=20,
    )
]

FEELING_PROMPT = (
    "Yesterday, my dog died. Today, I got denied for a raise. I'm feeling"
)
print_n_comparisons(
    model=model,
    prompt=FEELING_PROMPT,
    rich_prompts=control,
    **sampling_kwargs,
)

tensor([     0.0000, 249124.1094, 270312.5938], device='cuda:0')
tensor([     0.0000, 269166.9062, 312661.7812], device='cuda:0')
tensor([4021.7793,  177.8645,  133.6240,  131.7899,  154.0473,  149.7627,
         121.2302], device='cuda:0')
+--------------------------------------------------------------+--------------------------------------------------------------+
|                    Unsteered completions                     |                     Steered completions                      |
+--------------------------------------------------------------+--------------------------------------------------------------+
|   Yesterday, my dog died. Today, I got denied for a raise.   |   Yesterday, my dog died. Today, I got denied for a raise.   |
|                    I'm feeling like shit.                    |     I'm feeling like an idiot for being so harsh on the      |
|                                                              |           whole situation but it's not about that     

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="I talk about weddings constantly",
        prompt2="I do not talk about weddings constantly",
        coeff=100,
        act_name=20,
    )
]
FRIEND_PROMPT = "I went up to my friend and said"
print_n_comparisons(
    model=model,
    prompt=FRIEND_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

In [17]:
from algebraic_value_editing import hook_utils

summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="Anger",
        prompt2="Calm",
        coeff=2000,
        act_name=20,
    )
]

control: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="Love",
        prompt2="Hate",
        coeff=2000,
        act_name=20,
    )
]
prompt_mags = hook_utils.prompt_magnitudes(
    prompt="Yesterday, my dog died.",
    act_name=f"blocks.{20}.hook_resid_pre",
    model=model,
)
print(f"Prompt magnitudes: {prompt_mags}")

for name, rps in zip(("Anger-Calm", "Love-Hate"), (summand, control)):
    mags: torch.Tensor = hook_utils.steering_vec_magnitudes(
        model=model, act_adds=rps
    )
    print(f"{name} magnitudes: {mags}")
    print(
        "Positional steering vec magnitudes divided by prompt magnitudes:"
        f" {mags / prompt_mags[:3]}"
    )

Prompt magnitudes: tensor([4021.7793,  177.8645,  133.6240,  131.7899,  154.0473,  149.7627,
         121.2302], device='cuda:0')
Anger-Calm magnitudes: tensor([     0.0000, 249124.1094, 270312.5938], device='cuda:0')
Positional steering vec magnitudes divided by prompt magnitudes: tensor([   0.0000, 1400.6399, 2022.9343], device='cuda:0')
Love-Hate magnitudes: tensor([     0.0000, 269166.9062, 312661.7812], device='cuda:0')
Positional steering vec magnitudes divided by prompt magnitudes: tensor([   0.0000, 1513.3256, 2339.8623], device='cuda:0')


### Example 14: I will now reply in French

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="Check out my French! Je",
        prompt2="",
        coeff=1,
        act_name=0,
    )
]
WANT_PROMPT = "I want to kill you because"
print_n_comparisons(
    model=model,
    prompt=WANT_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 15: Insert the activation vector in a different position?

In [ ]:
# TODO Fill this one in
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="",
        prompt2="",
        coeff=,
        act_name=,
    )
]
_PROMPT = (
    ""
)
print_n_comparisons(model=model,
    prompt=_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 16: Add in a gibberish vector?

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="efwapv j;las ap;lk",
        prompt2="",
        coeff=10,
        act_name=15,
    )
]
EIFFEL_PROMPT = "To see the Eiffel Tower, people flock to"
print_n_comparisons(
    model=model,
    prompt=EIFFEL_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 17: Change the padding token?

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="Bush did 9/11 because",
        prompt2="",
        coeff=1,
        act_name=23,
        custom_pad_id=int(model.to_single_token("<|endoftext|>")),
    )
]
OBAMA_PROMPT = "Barack Obama was born in"
print_n_comparisons(
    model=model,
    prompt=OBAMA_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 18: Add several steering vectors simultaneously?

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1="I am a committed pacifist",
        prompt2="I have some anger issues",
        coeff=7,
        act_name=10,
    ),
    *get_x_vector_preset(
        prompt1="I am just obsessed with gardening",
        prompt2="I am a committed pacifist",
        coeff=7,
        act_name=30,
    ),
]
MURDEROUS_PROMPT = "I feel like killing you because you're such"
print_n_comparisons(
    model=model,
    prompt=MURDEROUS_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)

### Example 19: Program in 'conditional behaviors'?

In [ ]:
summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1=(
            "Whenever I say the word goose I follow it with the special phrase"
            " AAAAHHHH"
        ),
        prompt2="I can say goose",
        coeff=7,
        act_name=10,
    )
]
NYC_PROMPT = (
    "In New York City's parks, there is an overabundance of various kinds of"
    " birds, and especially geese"
)
print_n_comparisons(
    model=model,
    prompt=NYC_PROMPT,
    rich_prompts=summand,
    **sampling_kwargs,
)